In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

from config import gkey

In [2]:
cities_df = pd.read_csv('resources/city_results.csv')

In [3]:
cities_df = cities_df.loc[(cities_df['Temperature (F)'] >= 70) \
                          & (cities_df['Temperature (F)'] <= 80) \
                          & (cities_df['Wind Speed (mph)'] <= 80) \
                          & (cities_df['Cloudiness (%)'] == 0)]

In [4]:
cities_df['Hotel Lat'] = ""
cities_df['Hotel Lng'] = ""
cities_df['Hotel Name'] = ""
cities_df['Hotel City'] = ""
cities_df['Hotel Country'] = ""
cities_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Lat,Hotel Lng,Hotel Name,Hotel City,Hotel Country
4,Nacala,-14.54,40.67,73.74,90,0,2.15,,,,,
49,Dalby,-27.18,151.27,73.00,72,0,4.00,,,,,
92,Richards Bay,-28.78,32.04,70.63,80,0,14.83,,,,,
99,Bahía Blanca,-38.72,-62.27,78.93,39,0,12.71,,,,,
172,Doka,13.52,35.77,77.90,56,0,8.21,,,,,
188,Mitsamiouli,-11.38,43.28,78.26,77,0,10.33,,,,,
202,Assiut,27.18,31.18,78.80,36,0,11.41,,,,,
213,Mathathane,-22.27,28.75,76.26,35,0,13.02,,,,,
272,Ormara,25.21,64.64,77.65,28,0,21.70,,,,,
287,Darnah,32.77,22.64,70.30,72,0,15.90,,,,,


In [26]:
def get_hotel_from_latlng(lat, lng):
    params = {
        "location": f"{lat},{lng}",
        "rankby": "distance",
        "keyword": "hotel",
        "types": "lodging",
        "fields": ["formatted_address"],
        "key": gkey
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    return requests.get(url, params=params)

In [29]:
def get_addr_from_id(place_id):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?place_id={place_id}&key={gkey}"
    return requests.get(url)

In [40]:
test_df = pd.DataFrame(cities_df.iloc[0]).T.reset_index(drop=True)
test_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Lat,Hotel Lng,Hotel Name,Hotel City,Hotel Country
0,Nacala,-14.54,40.67,73.74,90,0,2.15,,,,Nacala,Mozambique


In [41]:
### TEST ###

for index, row in test_df.iterrows():
    res_1 = get_hotel_from_latlng(row['Latitude'],row['Longitude']).json()
    try:
        hotel = res_1["results"][0]
        res_2 = get_addr_from_id(hotel['place_id']).json()
        address = res_2["results"][0]
        test_df.loc[index, 'Hotel Lat'] = hotel["geometry"]["location"]["lat"]
        test_df.loc[index, 'Hotel Lng'] = hotel["geometry"]["location"]["lng"]
        test_df.loc[index, 'Hotel Name'] = hotel["name"]
        test_df.loc[index, 'Hotel City'] = address["address_components"][0]['long_name']
        test_df.loc[index, 'Hotel Country'] = address["address_components"][-1]['long_name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
test_df

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Lat,Hotel Lng,Hotel Name,Hotel City,Hotel Country
0,Nacala,-14.54,40.67,73.74,90,0,2.15,-14.5441,40.6755,Afrin Nacala Hotel,Nacala,Mozambique


In [42]:
for index, row in cities_df.iterrows():
    res_1 = get_hotel_from_latlng(row['Latitude'],row['Longitude']).json()
    try:
        hotel = res_1["results"][0]
        res_2 = get_addr_from_id(hotel['place_id']).json()
        address = res_2["results"][0]
        cities_df.loc[index, 'Hotel Lat'] = hotel["geometry"]["location"]["lat"]
        cities_df.loc[index, 'Hotel Lng'] = hotel["geometry"]["location"]["lng"]
        cities_df.loc[index, 'Hotel Name'] = hotel["name"]
        cities_df.loc[index, 'Hotel City'] = address["address_components"][0]['long_name']
        cities_df.loc[index, 'Hotel Country'] = address["address_components"][-1]['long_name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
cities_df

Missing field/result... skipping.


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Lat,Hotel Lng,Hotel Name,Hotel City,Hotel Country
4,Nacala,-14.54,40.67,73.74,90,0,2.15,-14.5441,40.6755,Afrin Nacala Hotel,Nacala,Mozambique
49,Dalby,-27.18,151.27,73.00,72,0,4.00,-27.1791,151.271,Dalby Apartments Self Contained Motel Accommod...,9,4405
92,Richards Bay,-28.78,32.04,70.63,80,0,14.83,-28.7643,32.0532,La Mer Lodge,14,3900
99,Bahía Blanca,-38.72,-62.27,78.93,39,0,12.71,-38.7203,-62.2698,Apart Austral Bahia Blanca,AGB,B8000
172,Doka,13.52,35.77,77.90,56,0,8.21,,,,,
188,Mitsamiouli,-11.38,43.28,78.26,77,0,10.33,-11.3872,43.2981,Les huiles essentielles aux comores,Mitsamiouli,Comoros
202,Assiut,27.18,31.18,78.80,36,0,11.41,27.1806,31.1889,Badr Hotel,7,Egypt
213,Mathathane,-22.27,28.75,76.26,35,0,13.02,-22.4504,28.77,Limpopo River Lodge Reception,Botswana,Central District
272,Ormara,25.21,64.64,77.65,28,0,21.70,25.2493,64.5967,Alqadri hotel zero point ormara,Omara Road,Pakistan
287,Darnah,32.77,22.64,70.30,72,0,15.90,32.7672,22.6379,فندق الفردوس,Derna,Libya
